Descobrindo a senha com DEAP
============================



## Importações



In [1]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

from funcoes import gene_letra

## Problema da senha usando `DEAP`



Primeiro definimos as constantes.



In [2]:
# relacionadas ao problema a ser resolvido
SENHA = "correcthorsebatterystaple"
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz"
NUM_GENES = len(SENHA)

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 500
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.15
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Precisamos definir a função objetivo também.



In [3]:
def funcao_objetivo_senha(individuo, senha_verdadeira):
    """Computa a funcao objetivo de um individuo no problema da senha

    Args:
      individiuo: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir

    Returns:
      Uma tupla com a  "distância" entre a senha proposta e a senha verdadeira.
      Essa distância é medida letra por letra. Quanto mais distante uma letra
      for da que deveria ser, maior é essa distância.
    """
    diferenca = 0

    for letra_candidato, letra_oficial in zip(individuo, senha_verdadeira):
        diferenca = diferenca + abs(ord(letra_candidato) - ord(letra_oficial))

    return (diferenca, )

O `DEAP` não tem uma função de mutação de letras, precisamos usar a que nós criamos na disciplina. Tem um pequeno porém, o `DEAP` espera que a função de mutação retorne um indivíduo dentro de uma tupla. Não sei dizer o motivo dessa escolha, temos que aceitar e seguir.



In [4]:
def mutacao_senha(individuo, letras):
    """Realiza a mutação de um gene no problema da senha.

    Args:
      individuo: uma lista representado um individuo no problema da senha
      letras: letras possíveis de serem sorteadas.

    Return:
      Uma tupla de um individuo (senha) com um gene mutado.
    """
    gene = random.randint(0, len(individuo) - 1)
    individuo[gene] = gene_letra(letras)
    return (individuo, )

O restante não tem segredo. Único detalhe é que agora o problema é de minimização, então precisamos alterar isso.



In [5]:
# Devemos alterar aqui por ser um problema de minimização
creator.create("Fitness_min", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness_min)

toolbox = base.Toolbox()

toolbox.register("cria_gene", gene_letra, LETRAS_POSSIVEIS)
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.cria_gene,
    NUM_GENES,
)
toolbox.register(
    "population", tools.initRepeat, list, toolbox.individual, TAMANHO_POP
)
toolbox.register("evaluate", funcao_objetivo_senha, SENHA)
toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register(
    "mutate",
    mutacao_senha,
    letras=LETRAS_POSSIVEIS
)

hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.population()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_CRUZAMENTO,
    mutpb=CHANCE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

print()
print("Hall da fama:")
print("".join(hall_da_fama.items[0]))

gen	nevals	avg  	std    	min	max
0  	50    	211.1	26.8777	138	272
1  	29    	184.66	21.1543	138	222
2  	30    	166.98	22.3736	118	238
3  	25    	150.16	15.2766	118	181
4  	29    	141.06	9.44544	118	170
5  	38    	135.96	7.48321	119	162
6  	34    	130.68	4.98975	116	138
7  	32    	126.2 	6.64229	106	141
8  	26    	120.4 	6.92532	106	131
9  	31    	113.32	7.02122	101	131
10 	34    	110.54	7.09989	95 	128
11 	34    	104.64	6.24103	95 	129
12 	36    	100.78	5.801  	90 	118
13 	26    	95.88 	3.98818	90 	113
14 	27    	93.36 	3.25429	88 	106
15 	39    	91.42 	2.94   	87 	103
16 	30    	89.88 	3.05706	80 	99 
17 	29    	88.92 	4.2607 	82 	107
18 	29    	86.4  	2.54558	82 	91 
19 	26    	85.26 	4.45336	80 	107
20 	35    	83.24 	4.06231	75 	94 
21 	28    	82.12 	4.75243	75 	98 
22 	32    	80.08 	4.87787	75 	99 
23 	31    	78.44 	3.93528	75 	93 
24 	32    	76.24 	1.77268	75 	81 
25 	30    	76.08 	3.04526	74 	89 
26 	19    	76.52 	3.56786	74 	92 
27 	23    	76.76 	4.63707	74 	96 
28 	27    	75.94